## Word Frequency
임의의 text file에 있는 word들의 빈도수를 구하려 한다. word는 대소문자 구분없고 숫자, 특수 문자들은 단어에서 배제된다.
따라서, word들의 list를 만들기 전에 file을 읽고 난 후
- 대문자는 소문자로 변환
- 숫자, 특수문자는 `' '` 로 변환해야 할 것이다.

#### Hint:
주어진 text를 한 번 scan으로 효율적으로 변환해 주는 string method를 사용하면 될 것이다.
`maketrans` method는 변환시키는 dictionany를 정의해 주고, `translate` method는 이를 가지고 변환한 새로운 string을 generate한다.

In [1]:
the_text = '"Well, I never!", said Alice.'
my_substitutions = the_text.maketrans(
  # If you find any of these
  "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!\"#$%&()*+,-./:;<=>?@[]^_`{|}~\\\r\n",
  # Replace them by these
  "abcdefghijklmnopqrstuvwxyz                                           ")

# Translate the text now.
cleaned_text = the_text.translate(my_substitutions)
print(cleaned_text)

 well  i never    said alice 


#### Input: 
인터넷에 있는 *Alice in Wonderland* 동화책 내용을 다음과 같이 fetch한다.

In [2]:
import urllib
url = "http://openbookproject.net/thinkcs/python/english3e/_downloads/alice_in_wonderland.txt" 
with urllib.request.urlopen(url) as f:
    contents = f.read().decode()
url_word = "http://openbookproject.net/thinkcs/python/english3e/_downloads/vocab.txt" 
with urllib.request.urlopen(url_word) as f:
    word_contents = f.read().decode()

In [3]:
new_contents = contents.translate(my_substitutions)
new_word_contents = word_contents.translate(my_substitutions)

### 분류 방법
 's 'd 는 is, has 와 would, had 와 구분이 어렵기 때문에 모두 한 단어 처리하였습니다.--done    
   
 'tis 는 it is의 영국식 줄임말이기 때문에, it, is 두 단어로 처리하였습니다.--done  
   
 shan't는 shall not의 영국식 줄임말이기 때문에, shall, not 두 단어로 처리하였습니다.--done  
   
 ~n't형은 최대한 ~과 not으로 구분하였습니다.--done  
   
또한, chapter를 셀때 사용한 로마문자 (I, II, III, IIII, V...XI)는 단어에서 제거하였습니다.--done  
  
I 같은 경우에는 "나"를 뜻할때도 많이 사용하기 때문에, 최종으로 나온 I count에서 -1 하였습니다.--done  
  
'em 은 them 의 줄임말이기 때문에 them으로 편입하였습니다.--done  
  
내용중 beau---ootiful은 beautiful로, soo--oop는 soup로,  e--e--evening은 evening으로,  동일한 패턴의 반복이므로 합쳐서 한 글자로 처리하였습니다.--done  
  
ied 과거형을 y로 변경했습니다. tie, die 같은 경우에는 ty, dy 로 없는 단어가 되어 tie, die로 바꾸어주었습니다.--done  
  
ed와 ing를 사용하기 위해서 사전에 제공해주신 단어장을 이용하였습니다. 완벽하지는 않으나, 어느정도 단어개수를 줄여줄 수 있을것으로 예상됩니다.  
  
ied 과거형을 y로 바꾸었습니다.--done  
  
ed 과거형을 현재형으로 최대한 바꾸었습니다.--done  
  
ed/d가 붙음으로인해 동사인지 형용사인지 혼동되는 부분이 있어서, 이부분에 대해 확실하게 구분을 하지 못했습니다.  
  
ed가 붙는 경우 사전에 주어진 단어장을 통해 형용사 먼저 단어에 추가하였습니다.  
  
순서: 형용사 -> ied -> ed -> ~d 순서대로 걸렀습니다.
  
ing 진행형을 현재형으로 최대한 바꾸었습니다.--done  

In [4]:
words = new_contents.split(" ")
word_list = new_word_contents.split(" ")

In [5]:
import copy
import string
save,align_word, buff = 'a', [], []
for word in word_list:
    if word is not "":
        if save != word[0]:
            align_word.append(copy.copy(buff))
            buff.clear()
        save = word[0]
        buff.append(word)
align_word.append(copy.copy(buff))

In [6]:
past_word = [[word+'ed' for word in align_word[i] if word is not ""]for i in range(26)]
d_word = [[word+'d' for word in align_word[i] if word is not ""]for i in range(26)]
ing_word = [[word+'ing' for word in align_word[i] if word is not ""]for i in range (26)]

주어진 사전을 이용해서 알파벳순으로 정렬된 list를 새로 만들었습니다.  
  
예) align_word[0] -> 알파벳 a에 관련된 단어  
  
이후, ~ed, ~d, ~ing를 구분하기위해  
  
모든 단어에 ed, d, ing를 붙인 새로운 list를 만들었습니다.  
  
조금 역발상으로, 단순히 ed가 붙는 단어를 ed를 제거하고 align_word에서 비교하는것 보다 align_word에 ed를 붙여 비교하는것이 더 정확하다고 생각이 들어서 이 방법을 채택했습니다. d 와 ing 또한 마찬가지입니다.  

In [7]:
def alpha_to_idx(str):
    idx = ord(str)-97
    return idx

In [8]:
import time

def add_to_dict(dict,str):
    dict.setdefault(str, 0)
    dict[str] += 1
count = 0
except_word = ['beau','ootiful','soo','oop','e','evening','beauti','ful']
chapter_rome = ['ii','iii','iv','v','vi','vii','viii','ix','x','xi','xii']
ed_word=['red','led','tired','hundred', 'indeed', 'speed', 'proceed']
empty_word, word_dict, except_list= ['', "'"], {}, []

for word in words:
    
#저는 maketrans method에서 '를 제외하고 공백으로 만들었기 때문에, 남아있는 '들을 처리하였습니다.

    if word in empty_word+chapter_rome:
        continue
        
#이후, 단어 앞 뒤에 '가 붙어있을 수 있기 때문에(대화형 말투 ex: 'Alice is angry.'에서 /'Alice/is/angry/'/이렇게 나뉘어짐)
#처리를 해주었습니다.

    if word[0] == "'":          #this part becase of ' character in the first line. ex) 'Aliece, 'He ...etc
        word = word[1:]
    elif word[-1] == "'":
        word = word[:-1]        #this part because of ' characters can be with in the word.
        
#em 같은 경우는 'em에서 앞에 '가 사라진 것이고, 검색결과 them의 줄임말이므로 바꾸어주었습니다.

    if word == 'em':            #cus upper case already removed ' form the first character, so only em is left in 'em    
        add_to_dict(word_dict, 'them')
        continue
        
#tis 또한 it is의 줄임말이기 때문에 나누어 주었습니다.

    if word == "'tis" or word == 'tis':          #'tis is abbreviation of it is.
        add_to_dict(word_dict, 'it')
        add_to_dict(word_dict, 'is')
        continue
        
#shan't, won't can't 모두 shall, will, can의 형태가 규칙적이지 않아 위에서 전처리하였습니다.

    if word == "shan't":        #shan't is abbreviation of shall not
        add_to_dict(word_dict, 'shall')
        add_to_dict(word_dict, 'not')        
        continue
    if word == "won't":         #won't is abbreviation of will not
        add_to_dict(word_dict, 'will')
        add_to_dict(word_dict, 'not')        
        continue        
    if word == "can't":         #can't is abbreviation of can not
        add_to_dict(word_dict, 'can')
        add_to_dict(word_dict, 'not')
        continue
        
#beau---ootiful soo--oop e--e--evening을 처리하기위해, 이 단어들을 예외단어list를 만들어두고, 후에 처리합니다.

    if word in except_word:     #declared exception word in upside of code to take care of special case.
        except_list.append(word)
        continue
        
#여기서 부터는 not, am, will, are, have가 축약되어있는 단어들을 처리하여줍니다.

    if "'" in word:             #normal cases include character ' in the word.
        if word[-2] == "'":     #can be 's, 'd, 't
            if word[-1] =='t':  #is going to be (normally) verb + not == verb+n't style
                add_to_dict(word_dict, word[:-3])
                add_to_dict(word_dict, 'not')
                continue
            elif word[-1] == 'm':       #to take care of I'm 
                add_to_dict(word_dict, word[:-2])
                add_to_dict(word_dict, 'am')
                continue
        if word[-3] == "'":
            if word[-3:] == "'ll":      #abbreviation of (something||somebody) will
                add_to_dict(word_dict, word[:-3])
                add_to_dict(word_dict, 'will')
                continue
            elif word[-3:] == "'re":    #abbreviation of (multiple) are
                add_to_dict(word_dict, word[:-3])
                add_to_dict(word_dict, 'are')
                continue 
            elif word[-3:] == "'ve":    #abbreviation of (something||somebody) have
                add_to_dict(word_dict, word[:-3])
                add_to_dict(word_dict, 'have')
                continue 
                
#어느정도 특수단어들이 끝난 이후에, 사전에 만들어둔 단어list를 통하여 해당 단어가 단어list에 있는지 검사합니다.
#과거형과 형용사가 ~ed로 중복되는 경우에는 형용사를 선택하기로 하였습니다.

    if word in align_word[alpha_to_idx(word[0])]:  #if the word is in dictionary, add to word_dict
        add_to_dict(word_dict, word)
        continue

#이후 과거형을 다룹니다. 과거형과 형용사가 ~ed로 중복되는 경우에는 형용사를 선택하기로 하였습니다.
#for here try to take care of past tens
    #~y 단어가 ed를 만나 ied가 되는 특수한 상황에 대해 처리하였습니다.

    if "ied" in word[-3:]:              #to take care of past tense, special case y->ied
        n_word = word[:-3] + 'y'
        #tie와 die같은경우에는 예외로 처리하여 주었습니다.
        if n_word == 'ty':              #special case of ied past tense. tied, died, ...etc
            add_to_dict(word_dict, 'tie')
            continue
        elif n_word == 'dy':
            add_to_dict(word_dict, 'die')
            continue
        add_to_dict(word_dict, n_word)
        continue
    
    #이후 ed의 형태, ~d 의 형태의 과거형을 다루었습니다.

    if 'ed' in word[-2:]:
        if word in past_word[alpha_to_idx(word[0])]:   #past tense, word+ed style.
            add_to_dict(word_dict, word[:-2])
            continue
        elif word in d_word[alpha_to_idx(word[0])]:    #this part is for past tense like liked, moved, closed... ~d style
            add_to_dict(word_dict,word[:-1])
            continue
        
        #이후 stopped 처럼 마지막 단어가 2번 중복해서 들어가는 단어들에 대해서 다루었습니다.
        
        elif len(word)>=4 and word[-3] == word[-4]:    #this part is for take care of like stopped, occurred... double letters
            add_to_dict(word_dict, word[-3])           #in the past tense.
            continue

#여기서 부터는 현재진행형을 현재형으로 바꿉니다.
#for here try to take care of ~ing style

    if 'ing' in word[-3:]:
        #우선 예외사항으로 atheling이라는 단어는 왕자를 뜻하는 명사이기에 제거하였습니다.
        if word == 'atheling':    #noun of prince
            add_to_dict(word_dict, word)
            continue
        word = word[:-3]
        
        #ing를 제거하고 곧바로 단어가 완성되는 단어에 대해 처리하였습니다.
        
        if word in align_word[alpha_to_idx(word[0])]:
            add_to_dict(word_dict, word)
            continue
            
        #ing를 더할때 맨 마지막 e를 생략하는 경우를 처리하였습니다.

        if word+'e' in align_word[alpha_to_idx(word[0])]: #word-e+ing style
            add_to_dict(word_dict, word+'e')
            continue
        
        #나머지 단어들중 마지막 단어가 중복되는 단어들을 처리하였습니다.
        
        elif word == 'shill': #exception of index -1 and -2 is same
            add_to_dict(word_dict, word)
            continue
        elif word[-1] == word[-2]:   #exception of index -1 and -2 is same
            add_to_dict(word_dict,word[:-1])
            continue
    add_to_dict(word_dict, word)
    
#여기서부터는 아까 추후 처리를 하려고 빼두었던 단어에 대해서 합성합니다.

prev, preprev = '',''
for i in except_list:        #to take care of beau ootiful and soo oop and e e evening
    if prev+i == 'beauootiful':
        add_to_dict(word_dict,'beautiful')
    elif prev+i == 'beautiful':
        add_to_dict(word_dict,'beautiful')
    elif prev+i == 'soooop':
        add_to_dict(word_dict,'soup')
    elif preprev+prev+i == 'eeevening':
        add_to_dict(word_dict,'evening')
    elif i == 'evening':
        add_to_dict(word_dict,'evening')
    prev = i
    preprev = prev
#위에 언급했다시피 로마숫자 I는 단어에서 제거하기로 하였기에, 하나 낮추어줍니다.
word_dict['i'] -= 1        #this is for I(number)


#### Q1. How many different words are used in the *Alice in Wonderland*?

In [9]:
print(len(word_dict))

2329


#### Q2. List top 20 frequently used words and their frequencies in the *Alice in Wonderland*.

In [10]:
word_rank = sorted(word_dict.items(), key = lambda item: item[1], reverse=True)
print(word_rank[:20])

[('the', 1643), ('and', 872), ('to', 729), ('a', 632), ('she', 544), ('it', 543), ('i', 533), ('of', 514), ('said', 462), ('you', 401), ('alice', 386), ('in', 369), ('was', 368), ('not', 362), ('that', 281), ('as', 263), ('her', 248), ('at', 212), ('on', 193), ('had', 186)]


#### Q3. 
As children learn to read, there are expectations that their vocabulary will grow. So a child of age 14 is expected to know more words than a child of age 8. When prescribing reading books for a grade, an important question might be “which words in this book are not in the expected vocabulary at this level?”

Find the words in the book *Alice in the Wonderland* are not in the vocabulary given in the file  http://openbookproject.net/thinkcs/python/english3e/_downloads/vocab.txt.

(어린이가 수준 이상이 되는 단어들을 찾아내는 문제다. 적절한 수준의 단어들로 채워진 단어장에 없으면 적정 수준을 초과한 어려운 단어라는 의미다.)

In [11]:
hard_word=[]
for word in word_dict:
    if word in align_word[alpha_to_idx(word[0])]:
        continue
    hard_word.append(word)
print(hard_word)
print("\nNumber of Hard word: ",len(hard_word))

["alice's", 'adventures', 'wonderland', 'lewis', 'carroll', 'alice', 'pictures', 'conversations', 'daisies', 'eyes', 'r', 'ought', 'waistcoat', 'p', 'sides', 'cupboards', 'maps', 'pegs', 'l', 'centre', 'learnt', 'lessons', 'schoolroom', "that's", 'words', 'antipathies', "ma'am", 'zealand', 'curtsey', 'dinah', 'cats', 'bats', 'ears', "it's", 'longer', 'lamps', 'doors', 'g', 'locks', 'inches', 'larger', 'loveliest', 'beds', 'flowers', 'fountains', 'shoulders', 'rules', 'telescopes', 'letters', 'histories', 'beasts', 'friends', 'bleeds', 'sooner', 'flavour', 'legs', "there's", 'tears', 'currants', 'makes', 'smaller', 'happens', 'eats', 'curiouser', 'largest', 'shoes', 'stockings', 'dears', 'boots', 'presents', "one's", 'esq', 'hearthrug', 'gallons', 'gloves', 'skurry', 'ada', 'goes', 'ringlets', 'mabel', 'sorts', 'knows', "she's", "let's", 'london', 'paris', 'rome', 'doth', 'hands', 'nile', 'seems', 'claws', 'fishes', 'toys', "rabbit's", 'railway', 'machines', 'houses', 'nearer', 'o', "br

참고사항)  
is 와 are가 다른 단어 인거 처럼 daisy와 daisies도 다른 단어로 취급하겠습니다. 단수,복수 구분해주세요 

"'s" 는 예외가 많습니다. (it has/ it is/ 소유격)
- 따라서, it's 를 그냥 새로운 한 단어로 생각해서 풀어주세요.
가령 girls' 라고 하면 이것도 새로운 한 단어라고 생각해주세요.
- Alice랑 Alice's 각각 새로운 한단어라고 생각하시면 됩니다. 만약, there's 와 there is 가 나왔으면 전자는 1단어 후자는 2단어 입니다.